In [1]:
import cv2
import mediapipe as mp
import random
import time

# Configuraciones de MediaPipe para la detección de manos
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

# Inicializar la captura de video de la cámara
cap = cv2.VideoCapture(0)

# Variables para el juego
gestos = ["Piedra", "Papel", "Tijera"]
puntaje_usuario = 0
puntaje_maquina = 0
rondas_jugadas = 0

# Pantalla de bienvenida
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Mostrar un mensaje de bienvenida
    mensaje_bienvenida = "Bienvenido a Piedra, Papel, Tijera!"
    instrucciones = "Presiona 's' para comenzar el juego"
    
    # Mostrar el mensaje de bienvenida
    cv2.putText(frame, mensaje_bienvenida, (50, frame.shape[0] // 2 - 50), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 215, 255), 3, cv2.LINE_AA)
    cv2.putText(frame, instrucciones, (50, frame.shape[0] // 2 + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2, cv2.LINE_AA)

    # Mostrar el frame en pantalla
    cv2.imshow('Piedra, Papel, Tijera - Bienvenida', frame)

    # Esperar a que el usuario presione 's' para empezar
    if cv2.waitKey(1) & 0xFF == ord('s'):
        break

# Cuenta regresiva inicial antes de comenzar el juego
tiempo_inicio_cuenta = time.time()
duracion_cuenta = 3  # Duración de la cuenta regresiva en segundos

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Calcular el tiempo restante de la cuenta regresiva
    tiempo_transcurrido = int(time.time() - tiempo_inicio_cuenta)
    tiempo_restante = duracion_cuenta - tiempo_transcurrido

    if tiempo_restante <= 0:
        break  # Finalizar la cuenta regresiva

    # Mostrar el mensaje de cuenta regresiva
    mensaje_cuenta_regresiva = f"Comenzamos en {tiempo_restante}..."
    cv2.putText(frame, mensaje_cuenta_regresiva, (frame.shape[1] // 2 - 150, frame.shape[0] // 2), 
                cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 215, 255), 4, cv2.LINE_AA)

    # Mostrar el frame en pantalla
    cv2.imshow('Piedra, Papel, Tijera - Cuenta Regresiva', frame)

    # Salir si el usuario presiona 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Iniciar la detección de manos con MediaPipe
ultimo_tiempo = time.time()
gesto_maquina = random.choice(gestos)
puntaje_actualizado = False  # Bandera para controlar la actualización de puntaje
tiempo_cambio_gesto = 4  # Tiempo en segundos para cambiar el gesto de la máquina

with mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.5) as hands:
    while cap.isOpened():
        # Verificar si el juego ha alcanzado el número máximo de rondas
        ret, frame = cap.read()
        if not ret:
            break
        
        if puntaje_usuario == 3 or puntaje_maquina == 3:
            break

        # Inicializar el resultado para evitar errores de referencia antes de ser definido
        resultado = ""

        # Convertir la imagen de BGR a RGB para MediaPipe
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(frame_rgb)

        # Tiempo restante para el cambio del gesto de la máquina
        tiempo_restante = int(tiempo_cambio_gesto - (time.time() - ultimo_tiempo))

        # Variable para el gesto del usuario
        gesto_usuario = None

        # Si se detecta una mano en la imagen
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                # Dibujar las marcas en la mano
                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                # Obtener la posición de las marcas relevantes en los dedos
                thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP].y
                index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y
                middle_tip = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP].y
                ring_tip = hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_TIP].y
                pinky_tip = hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_TIP].y

                # Detectar el gesto del usuario
                if (index_tip < thumb_tip and middle_tip < thumb_tip and
                        ring_tip < thumb_tip and pinky_tip < thumb_tip):
                    gesto_usuario = "Papel"
                elif (index_tip < thumb_tip and index_tip < ring_tip and
                        middle_tip < thumb_tip and middle_tip < ring_tip and
                        ring_tip > index_tip and pinky_tip > index_tip):
                    gesto_usuario = "Tijera"
                elif (index_tip > thumb_tip and middle_tip > thumb_tip and
                        ring_tip > thumb_tip and pinky_tip > thumb_tip):
                    gesto_usuario = "Piedra"

        # Solo mostrar y comparar el gesto después de que el tiempo de cuenta regresiva llegue a 0
        if tiempo_restante == 0:
            # Actualizar el gesto de la máquina y resetear el tiempo
            if not puntaje_actualizado:
                gesto_maquina = random.choice(gestos)
                puntaje_actualizado = True  # Evitar que se actualice hasta la próxima ronda

                # Comparar el gesto del usuario con el de la máquina
                if gesto_usuario:
                    if gesto_usuario == gesto_maquina:
                        resultado = "Empate"
                    elif (gesto_usuario == "Piedra" and gesto_maquina == "Tijera") or \
                            (gesto_usuario == "Papel" and gesto_maquina == "Piedra") or \
                            (gesto_usuario == "Tijera" and gesto_maquina == "Papel"):
                        resultado = "Usuario gana"
                        puntaje_usuario += 1
                    else:
                        resultado = "Maquina gana"
                        puntaje_maquina += 1

                # Incrementar la cantidad de rondas jugadas
                rondas_jugadas += 1

                # Resetear el tiempo para la próxima ronda
                ultimo_tiempo = time.time()
                puntaje_actualizado = False

        # Mostrar el marcador de puntos con un estilo moderno
        marcador_ancho = 220
        marcador_alto = 150
        color_panel = (50, 50, 50)
        color_texto = (255, 255, 255)
        grosor_borde = 2

        # Rectángulo para el puntaje del usuario
        cv2.rectangle(frame, (20, 20), (20 + marcador_ancho, 20 + marcador_alto), color_panel, -1)
        cv2.putText(frame, "USUARIO", (35, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color_texto, 2, cv2.LINE_AA)
        cv2.putText(frame, f"{puntaje_usuario}", (100, 130), cv2.FONT_HERSHEY_SIMPLEX, 2, color_texto, 3, cv2.LINE_AA)

        # Rectángulo para el puntaje de la máquina
        cv2.rectangle(frame, (300, 20), (300 + marcador_ancho, 20 + marcador_alto), color_panel, -1)
        cv2.putText(frame, "MAQUINA", (315, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color_texto, 2, cv2.LINE_AA)
        cv2.putText(frame, f"{puntaje_maquina}", (380, 130), cv2.FONT_HERSHEY_SIMPLEX, 2, color_texto, 3, cv2.LINE_AA)

        # Mostrar el resultado de la partida en el centro, con un fondo más moderno
        if resultado:
            resultado_ancho = 400
            resultado_alto = 80
            cv2.rectangle(frame, (frame.shape[1] // 2 - resultado_ancho // 2, frame.shape[0] // 2 - resultado_alto // 2), 
                          (frame.shape[1] // 2 + resultado_ancho // 2, frame.shape[0] // 2 + resultado_alto // 2), (0, 0, 0), -1)
            cv2.putText(frame, resultado, (frame.shape[1] // 2 - 150, frame.shape[0] // 2 + 10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 215, 255), 3, cv2.LINE_AA)
        
        # Mostrar el gesto del usuario y de la máquina en una posición más homogénea
        cv2.putText(frame, f"Gesto usuario: {gesto_usuario if gesto_usuario else 'No detectado'}", (20, 200), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 0), 2, cv2.LINE_AA)
        cv2.putText(frame, f"Gesto maquina (Ronda anterior): {gesto_maquina}", (20, 240), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 0), 2, cv2.LINE_AA)

        # Mostrar la cuenta regresiva visualmente con un círculo en el centro de la pantalla
        centro_x, centro_y = frame.shape[1] // 2, frame.shape[0] // 2
        radio = 100
        porcentaje_completado = tiempo_restante / tiempo_cambio_gesto
        angulo = int(360 * porcentaje_completado)
        color_circulo = (0, 255, 0)  # Verde para la cuenta regresiva

        # Dibujar el círculo de la cuenta regresiva
        if tiempo_restante > 0:
            cv2.ellipse(frame, (centro_x, centro_y), (radio, radio), -90, 0, angulo, color_circulo, 10)

        # Mostrar el video en tiempo real
        cv2.imshow('Piedra, Papel, Tijera - Deteccion de gestos', frame)

        # Salir al presionar la tecla 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# Pantalla de resumen de la partida
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Determinar quién ganó la partida
    if puntaje_usuario > puntaje_maquina:
        ganador = "Usuario ha ganado la partida!"
    else:
        ganador = "Maquina ha ganado la partida!"

    # Mostrar el mensaje del ganador en grande
    cv2.putText(frame, ganador, (frame.shape[1] // 2 - 300, frame.shape[0] // 2 - 50), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 215, 255), 5, cv2.LINE_AA)

    # Mostrar el resumen de la partida
    resumen = f"Resumen de la Partida"
    rondas = f"Rondas Jugadas: {rondas_jugadas}"
    usuario_gano = f"Rondas Ganadas por Usuario: {puntaje_usuario}"
    maquina_gano = f"Rondas Ganadas por Maquina: {puntaje_maquina}"
    instrucciones = "Presiona 'q' para salir"

    # Mostrar los textos del resumen
    cv2.putText(frame, resumen, (50, 150), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 215, 255), 3, cv2.LINE_AA)
    cv2.putText(frame, rondas, (50, 200), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2, cv2.LINE_AA)
    cv2.putText(frame, usuario_gano, (50, 250), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2, cv2.LINE_AA)
    cv2.putText(frame, maquina_gano, (50, 300), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2, cv2.LINE_AA)
    cv2.putText(frame, instrucciones, (50, 400), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 0), 2, cv2.LINE_AA)

    # Mostrar el frame en pantalla
    cv2.imshow('Piedra, Papel, Tijera - Resumen', frame)

    # Esperar a que el usuario presione 'r' para reiniciar o 'q' para salir
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break

# Liberar recursos
cap.release()
cv2.destroyAllWindows()


2024-11-15 16:37:20.271 python[5665:3477177] +[IMKClient subclass]: chose IMKClient_Legacy
2024-11-15 16:37:20.271 python[5665:3477177] +[IMKInputSession subclass]: chose IMKInputSession_Legacy
I0000 00:00:1731685046.253734 3477177 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M2
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1731685046.276440 3477988 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1731685046.283348 3477988 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1731685046.304079 3477986 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.
